# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

## Setup: Import what you'll need to scrape the page

We'll be using Selenium for this, *not* BeautifulSoup and requests.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [5]:
# using Firefox here, since Chrome for some reason doesn't display any fonts, 
# which gives me just colourful boxes and not much else, which is annoying
driver = webdriver.Firefox()
driver.get('https://www.tdlr.texas.gov/cimsfo/fosearch.asp')

In [7]:
license_select = Select(driver.find_element_by_name('pht_status'))
license_select.select_by_visible_text('Cosmetologists')

In [8]:
last_name_input = driver.find_element_by_name('pht_lnm')
last_name_input.send_keys('Nguyen')

In [9]:
submit_button = driver.find_element_by_name('B1')
submit_button.click()

## Scraping

Once you are on the results page, do this.

### Loop through each result and print the entire row

Okay wait, that's a heck of a lot. Use `[:10]` to only do the first ten (`listname[:10]` gives you the first ten).

In [11]:
table_rows = driver.find_elements_by_tag_name('tr')

for row in table_rows[:10]:
  print(row.text)

Name and Location Order Basis for Order
NGUYEN, TOAN HUU
City: SAN ANTONIO
County: BEXAR
Zip Code: 78217


License #(s): 780948, 1706491, 1699123

Complaint # COS20180004289 Date: 5/30/2018

Respondent is assessed an administrative penalty in the amount of $500. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.
NGUYEN, HANH CONG
City: EL PASO
County: EL PASO
Zip Code: 79934


License #: 737708

Complaint # COS20180006594 Date: 5/30/2018

Respondent is assessed an administrative penalty in the amount of $1,000. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.
NGUYEN, KHIEM VAN
City: LONGVIEW
County: GREGG
Zip Code: 75604


License #: 731665

Complaint # COS20180000257 Date: 5/17/2018

Respondent is assessed an administrative penalty in the amount of $1,250. Responde

NGUYEN, LAN T-THUY
City: SAN ANTONIO
County: BEXAR
Zip Code: 78255

NGUYEN, SAMLOI
City: SAN ANTONIO
County: BEXAR
Zip Code: 78255


License #: 767339

Complaint # COS20180009255 Date: 5/17/2018

Respondent is assessed an administrative penalty in the amount of $575. Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use.
NGUYEN, TUAN A
City: AUSTIN
County: TRAVIS
Zip Code: 78723

NGUYEN, TUAN VAN
City: ARLINGTON
County: TARRANT
Zip Code: 76011


License #: 681274

Complaint # COS20140018343 Date: 5/9/2018

Respondent is assessed an administrative penalty in the amount of $1,000. Respondent failed to clean and disinfect all wax pots; Respondent failed to disinfect tools, implements, and supplies with a hospital-grade EPA-registered disinfectant solution; Respondent failed to replace materials that contact skin, or skin products from multi-use containers, with new or clean articles for each client.
NGUYEN, THAO B
City: EULESS
County: TARRANT

### Loop through each result and print each person's name

You'll get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

In [13]:
for row in table_rows[:10]:
  try:
    name = row.find_element_by_css_selector('td:first-child '
                                            'span.results_text:first-child')
    print(name.text)
  except:
    pass


NGUYEN, TOAN HUU
NGUYEN, HANH CONG
NGUYEN, KHIEM VAN
NGUYEN, DIEP THI NGOC
NGUYEN, LAN T-THUY
NGUYEN, TUAN A
NGUYEN, THAO B
NGUYEN, BETH MARIA


NGUYEN, TRUNG N


## Loop through each result, printing each violation description ("Basis for order")

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: You can get the HTML of something by doing `.get_attribute('innerHTML')` - it might help you diagnose your issue.*
> - *Tip: Or I guess you could just skip the one with the problem...

In [15]:
for row in table_rows[1:10]:
  basis_for_order_element = row.find_element_by_css_selector('td:last-child')
  print(basis_for_order_element.text)

Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.
Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect all wax pots.
Respondent failed to disinfect tools, implements, and supplies with an EPA-registered disinfectant solution; Respondent failed to disinfect multi-use equipment, implements, and tools prior to use on each client.
Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use.
Respondent failed to clean and disinfect all wax pots; Respondent failed to disinfect tools, implements, an

## Loop through each result, printing the complaint number

- TIP: Think about the order of the elements

In [16]:
for row in table_rows[1:10]:
  complaint_number_element = row.find_element_by_css_selector('td:first-child span.results_text:last-child')
  print(complaint_number_element.text)

COS20180004289
COS20180006594
COS20180000257
COS20180004915
COS20180009255
COS20140018343
COS20180008846
COS20180000897
COS20170023893


## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

> *Tip: If you want to ask for the "next sibling," you can't use `find_next_sibling` in Selenium, you need to use `element.find_element_by_xpath("following-sibling::div")` to find the next div, or `element.find_element_by_xpath("following-sibling::*")` to find the next anything.

In [25]:
list = []

for row in table_rows[1:]:
  dict = {}

  # Person's name
  name_element = row.find_element_by_css_selector('td:first-child >'
                                                  'span.results_text:first-child')
  dict['name'] = name_element.text

  # - Violation description
  violation_element = row.find_element_by_css_selector('td:last-child')
  dict['description'] = violation_element.text

  # - Violation number
  violation_nr_element = row.find_element_by_css_selector(
    'td:first-child span.results_text:last-child')
  dict['violation_number'] = violation_nr_element.text

  # - License Numbers
  license_nrs_element = row.find_element_by_css_selector('td:first-child '
                                                         '.results_text:nth-last-of-type(3)')
  dict['license_numbers'] = license_nrs_element.text

  # - Zip Code
  zip_element = row.find_element_by_css_selector('td:first-child '
                                                 'span:nth-of-type(7)')
  dict['zip'] = zip_element.text

  # - County
  county_element = row.find_element_by_css_selector('td:first-child '
                                                    'span:nth-of-type(5)')
  dict['county'] = county_element.text

  # - City
  city_element = row.find_element_by_css_selector('td:first-child '
                                                  'span:nth-of-type(3)')
  dict['city'] = city_element.text

  list.append(dict)


### Save that to a CSV

- Tip: You'll want to use pandas here

In [26]:
import pandas as pd

In [29]:
df = pd.DataFrame(list)

In [31]:
df.to_csv('07-homework/violations.csv', index=False)

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.